# Medical LLM Fine-Tuning for Medicine Suggestions

This notebook fine-tunes a medical language model using the allopathy medicines dataset to improve medicine suggestion accuracy.

**Dataset**: 1,027 medicines with disease conditions, contraindications, side effects, and drug interactions.

**Approach**: Fine-tune a small medical LLM (BioGPT or similar) using LoRA/QLoRA for efficient training on Colab's free GPU.

## 1. Setup and Installation

In [ ]:
# Install required packages
!pip install -q transformers datasets peft accelerate bitsandbytes wandb trl
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
import torch
import pandas as pd
import json
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline
)
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2. Upload and Prepare Dataset

In [ ]:
# Upload your CSV file using Colab's file upload
from google.colab import files

print("Please upload allopathy_medicines_plus_1000.csv")
uploaded = files.upload()

# Load the dataset
df = pd.read_csv('allopathy_medicines_plus_1000.csv')
print(f"Loaded {len(df)} medicines")
df.head()

In [ ]:
# Create instruction-following dataset
def create_training_examples(row):
    """Convert medicine data into instruction-response pairs."""
    examples = []
    
    medicine_name = row.get('medicine_name', '') or row.get('generic_name', '')
    disease = row.get('disease_condition', '')
    category = row.get('therapeutic_category', '')
    contraindications = row.get('contraindications', '')
    side_effects = row.get('common_side_effects', '')
    interactions = row.get('drug_interactions', '')
    dosage_form = row.get('dosage_form', '')
    
    if not medicine_name:
        return examples
    
    # Example 1: Disease to medicine
    if disease:
        instruction = f"What medicine can be used for {disease}?"
        response = f"{medicine_name} is used for {disease}. "
        if category:
            response += f"It belongs to the {category} category. "
        if contraindications:
            response += f"⚠️ Contraindications: {contraindications}. "
        if side_effects:
            response += f"Common side effects: {side_effects}. "
        response += "Always consult a physician before taking any medication."
        
        examples.append({
            'instruction': instruction,
            'response': response
        })
    
    # Example 2: Medicine information
    instruction = f"Tell me about {medicine_name}"
    response = f"{medicine_name} is a medicine"
    if category:
        response += f" in the {category} category"
    if disease:
        response += f" used for {disease}"
    response += ". "
    if dosage_form:
        response += f"Available as {dosage_form}. "
    if contraindications:
        response += f"⚠️ Do not use if: {contraindications}. "
    if side_effects:
        response += f"Side effects: {side_effects}. "
    response += "Consult a physician for proper dosage."
    
    examples.append({
        'instruction': instruction,
        'response': response
    })
    
    # Example 3: Contraindications query
    if contraindications:
        instruction = f"What are the contraindications for {medicine_name}?"
        response = f"⚠️ {medicine_name} should NOT be used in the following cases: {contraindications}. Always consult a healthcare professional before taking this medication."
        
        examples.append({
            'instruction': instruction,
            'response': response
        })
    
    return examples

# Generate training examples
all_examples = []
for _, row in df.iterrows():
    all_examples.extend(create_training_examples(row))

print(f"Generated {len(all_examples)} training examples")

# Create dataset
train_data = Dataset.from_list(all_examples)
print("\nSample example:")
print(train_data[0])

In [ ]:
# Format for instruction tuning
def format_instruction(example):
    """Format as instruction-following prompt."""
    prompt = f"""### Instruction:
{example['instruction']}

### Response:
{example['response']}"""
    return {'text': prompt}

train_data = train_data.map(format_instruction)
print("\nFormatted example:")
print(train_data[0]['text'])

## 3. Load Base Model with QLoRA

In [ ]:
# Model selection - using a small medical/general LLM
# Options:
# - "microsoft/BioGPT-Large" - Medical domain
# - "meta-llama/Llama-2-7b-hf" - General (requires HF token)
# - "TinyLlama/TinyLlama-1.1B-Chat-v1.0" - Lightweight

model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"  # Fast training on free Colab

# QLoRA configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

model = prepare_model_for_kbit_training(model)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"Model loaded: {model_name}")
print(f"Model size: {model.get_memory_footprint() / 1e9:.2f} GB")

## 4. Configure LoRA

In [ ]:
# LoRA configuration
peft_config = LoraConfig(
    r=16,  # LoRA rank
    lora_alpha=32,  # LoRA alpha
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Attention layers
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

## 5. Training Configuration

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./medical-medicine-lora",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=2,
    logging_steps=10,
    save_steps=100,
    warmup_steps=50,
    lr_scheduler_type="cosine",
    optim="paged_adamw_8bit",
    report_to="none",  # Change to "wandb" if you want tracking
)

# SFT Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
)

print("Trainer configured successfully!")

## 6. Train the Model

In [ ]:
# Start training
print("Starting training...")
trainer.train()

print("\nTraining complete!")

## 7. Save the Model

In [ ]:
# Save LoRA adapters
model.save_pretrained("medical-medicine-lora-final")
tokenizer.save_pretrained("medical-medicine-lora-final")

print("Model saved to: medical-medicine-lora-final/")

# Download to local machine
!zip -r medical-medicine-lora-final.zip medical-medicine-lora-final/
files.download('medical-medicine-lora-final.zip')

## 8. Test the Fine-Tuned Model

In [ ]:
# Test inference
def generate_response(instruction):
    prompt = f"""### Instruction:
{instruction}

### Response:
"""
    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.7,
        top_p=0.9,
        do_sample=True
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Extract only the response part
    response = response.split("### Response:")[-1].strip()
    return response

# Test queries
test_queries = [
    "What medicine can be used for fever?",
    "Tell me about Paracetamol",
    "What are the contraindications for Ibuprofen?",
    "Suggest medicine for bacterial infection"
]

print("Testing fine-tuned model:\n")
for query in test_queries:
    print(f"Q: {query}")
    print(f"A: {generate_response(query)}")
    print("-" * 80)

## 9. Convert to GGUF for Local Use (Optional)

In [ ]:
# Install llama.cpp converter
!pip install -q llama-cpp-python

# Merge LoRA with base model
from peft import PeftModel

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)

merged_model = PeftModel.from_pretrained(base_model, "medical-medicine-lora-final")
merged_model = merged_model.merge_and_unload()

# Save merged model
merged_model.save_pretrained("medical-medicine-merged")
tokenizer.save_pretrained("medical-medicine-merged")

print("Merged model saved!")

# Convert to GGUF (requires llama.cpp)
# !python llama.cpp/convert.py medical-medicine-merged --outtype q4_k_m --outfile medical-medicine.gguf

## 10. Integration Instructions

### To use this model in your project:

1. **Download the model** from this notebook
2. **Upload to your backend** at `mm-hie-backend/models/medical-medicine-lora/`
3. **Update RAG engine** to use the fine-tuned model:

```python
# In app/rag/rag_engine.py
from peft import PeftModel

# Load fine-tuned model
base_model = AutoModelForCausalLM.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "./models/medical-medicine-lora")
```

### Benefits:
- ✅ **Accurate medicine suggestions** based on diseases
- ✅ **Contraindication awareness** - knows when NOT to suggest medicines
- ✅ **Side effects knowledge** - can warn about common side effects
- ✅ **Drug interaction awareness** - understands medicine interactions
- ✅ **Small model size** (~1-2GB) - runs on CPU/GPU efficiently